In [1]:
import requests
from pymongo import MongoClient
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import urllib2
import sys

In [2]:
# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.

# years = ["2015", "2016", "2017"]
# months = ["jan", "feb"]
# dates = [str(n) for n in xrange(14,20)]

years = ["2017"]
months = ["oct"]
dates = [str(31)]


# count of number of links
i = 0

# Initiating mongoDB. Using pymongo to connect the database
client = MongoClient()
db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database

result = db.urls.delete_many({}) # A fresh start to the DB table -> removing all entries 
all_links = []

for month in months:
    for date in dates:
        root_url = "https://www.theguardian.com/commentisfree/2016/" + month + "/" + date 
        
        # This one contains all the URLs. Soup extracts those.
        list_of_urls = requests.get(root_url).text
        soup = BeautifulSoup(list_of_urls, "html.parser")

        # the .findAll method from re finds the _content_ that has 
        # html tag - 'a', attribute - 'href' and 
        # the pattern that begins with the root_url variable
        for link in soup.findAll('a', attrs={'href': re.compile(root_url)}):
            all_links.append(link.get('href'))
            
all_links = set(all_links)
for link in all_links:
    # urls is the name of the mongo DataBase. insert_one is the method 
    urls.insert_one({"id" : i, "url" : link})
#             print link
    i += 1
print "Total number of links stored is ", i

Total number of links stored is  29


In [3]:

i = 0
for d in urls.find():
    i += 1
print "Total number of links in the database table called url is", i


Total number of links in the database table called url is 29


In [6]:
def download_comments(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    startTime = time.time()
    fileName = 'GuardianDownload_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
    text = ''

    print '[~] Start downloading pages ...'
    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
            html = response.read()
            length = len(text)
            text += html
            if len(text) <= length:
                print "Text appending isn't working as expected"
            else :
                print "Text appending may have worked"
            if downloadCount == 1:
                soup = BeautifulSoup(html, 'html.parser')
            if downloadCount %5 == 0:
                print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    stopTime = time.time()
    runningTime = (stopTime - startTime)/60;
    print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
    return BeautifulSoup(text, 'html.parser')

In [7]:
def num_shares(url):
    '''
    This function reads the url using the phanton browser, phantom because it won't open a window,
    read the webpage and find the number of shares of the link.
    '''
    
    
    driver = webdriver.PhantomJS('/media/sanhita/Windows7_OS/workspace/downloads/phantomjs-2.1.1-linux-x86_64/bin/phantomjs')
    driver.get(url) #navigate to the page using the code; it takes a while to run this call
    time.sleep(5)
    html_source = driver.page_source 
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'html.parser')
    for number in soup.find('div', {"class" : "sharecount__value--short"}):
        n = number
    if not number:            
        number = 0
    
    return int(n)
        
        


In [5]:
len(urls.distinct(key='url'))

29

In [ ]:
# To retrieve data from mongodb, using id as the identifier

for url in urls.find({'id': 0}):
    print url['url']

In [ ]:
urls.find({'id': 0})

In [4]:
num_shares('https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police')

NameError: name 'num_shares' is not defined

In [ ]:
url = "https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"

text = requests.get(url).text
soup = BeautifulSoup(text, "html.parser")
# print soup.prettify()

In [ ]:
print soup.prettify()

In [ ]:
url = "https://www.theguardian.com/commentisfree/2017/oct/23/houston-harvey-construction-workers-protect-abuse-undocumented"
url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'
soup = download_comments(url, "test")



In [ ]:
soup.prettify()

In [ ]:
comments = soup.findAll("div", { "class" : "d-comment__body" })
recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = soup.findAll("span", {"itemprop" : "givenName"})
replies = soup.findAll("span", {"class" : "d-comment__reply-to-author"})
links = soup.findAll("a", {"itemprop" : "d-comment__reply-to-author"})

i = 0
for user, comment, recommend, reply in zip(users, comments, recommends, replies):
    if i == 100:
        print user.text, comment.text, int(recommend.text), reply.text
    i += 1


In [ ]:
user_ids = soup.findAll("li", attrs_="data-comment-author-id"})
comment_ids = soup.findAll("li", {"id" : "data-comment-id"})
authors = soup.findAll("li", {"id" : "data-comment-author"})

len(comment_ids), len(user_ids), len(authors)

In [ ]:
i = 0
lst = []
for lis in soup.find_all('li'):
    if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
#         print lis.attrs['data-comment-author-id']
        i += 1

print  i

In [ ]:

for i in xrange (len(lst)):
    if 'data-comment-author-id' in lst[i]:
        print i


In [ ]:
int(recommend.text)

In [ ]:
len(replies), len(users), len(comments), len(recommends)

In [ ]:
print soup.find('li', attrs={'id': 'data-comment-author-id'})

In [ ]:
f = open("GuardianDownload_4fmtq_1508968524.96_test.html", 'r')
text = f.read()
f.close()

In [ ]:
s2 = BeautifulSoup(text, 'html.parser')
comments = s2.findAll("div", { "class" : "d-comment__body" })
recommends = s2.findAll("span", {"class" : "d-comment__recommend-count--old"})
users = s2.findAll("span", {"itemprop" : "givenName"})
replies = s2.findAll("span", {"class" : "d-comment__reply-to-author"})

len(replies), len(users), len(comments), len(recommends)

In [ ]:
url = "https://www.theguardian.com/us/commentisfree"

list_of_urls = requests.get(url).text

soup2 = BeautifulSoup(list_of_urls, "html.parser")


import re
i = 0
for link in soup2.findAll('a', attrs={'href': re.compile("https://www.theguardian.com/commentisfree/2017")}):
    print link.get('href')
    i += 1
print i

In [ ]:
url = 'https://www.theguardian.com/commentisfree/2016/jan/14/prevent-british-women-isis-syria-met-police'

In [ ]:
soup = BeautifulSoup(urllib2.urlopen(url).read(), 'html.parser')

In [ ]:
i = 0
lst = []
for lis in soup.find_all('a'):
    if 'data-link-name' in (lis.attrs) and 'submeta__link' in lis.attrs:
        print lis.attrs['data-link-name']
        i += 1

print  i


# for topic in soup.find_all("a", { "class" : "data-link-name" }):
#     print topic

In [ ]:
for topic in soup.findAll("a", {"class" : "submeta__link"}):
    if 'data-link-name' in topic.attrs and 'keyword: ' in topic.attrs['data-link-name']:
        print topic.attrs['data-link-name'][9:]
# print section


In [ ]:
set(section)

In [ ]:
for s in soup.find("h1", {"class" : "content__headline"}):
    print s

# Do not run the cell below this one

In [ ]:
# The one below is a github code for downloading comments. Do not use this; 
# instead use a modified version of this in the cell below this one.

import urllib2
import re
import time
import sys

# This script will download all comments from a Guardian (theguardian.com) article into a single .html file
# Use 'File ID' to easily identify your file later (this could also be something like a Citavi identifier)
# Ingo Kleiber <ikleiber@googlemail.com>
# https://gist.github.com/IngoKl/3ebd968ef3c6da406638

print """\
   _____                     _ _             _____  _      
  / ____|                   | (_)           |  __ \| |     
 | |  __ _   _  __ _ _ __ __| |_  __ _ _ __ | |  | | |     
 | | |_ | | | |/ _` | '__/ _` | |/ _` | '_ \| |  | | |     
 | |__| | |_| | (_| | | | (_| | | (_| | | | | |__| | |____ 
  \_____|\__,_|\__,_|_|  \__,_|_|\__,_|_| |_|_____/|______|
"""                                                         

print 'Guardian Comment Downloader v.1.1 - Ingo Kleiber (05.01.2015)\n'
guardianLink = raw_input ('> Guadian Link: ')
fileId = raw_input('> File ID: ')

#Figuring out the PageId
pageResponse = urllib2.urlopen(guardianLink)
commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
# print "Comments Match = ", commentsMatch()
if commentsMatch:
    pageId = commentsMatch.group(1)
    print '[+] pageId has been retrieved ('+pageId+')'
else:
    sys.exit('[-] Could not retrieve pageId!')

#Retrieving comments
downloadCount = 1
downloadError = 0

startTime = time.time()
fileName = 'GuardianDownload_' + pageId + '_' + str(startTime) + '_' + fileId + '.html'
f = open(fileName, 'a')


print '[~] Start downloading pages ...'
while downloadError == 0:
    try:
        response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId + '?page=' + str(downloadCount))
        html = response.read()
        f.write(html)
        if downloadCount == 1:
            soup = BeautifulSoup(html, 'html.parser')
        print '[+] Page: ' + str(downloadCount) + ' has been downloaded!'
        downloadCount = downloadCount + 1
        
    except:
        downloadError = 1

stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes' 		
f.close() 